In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import neighbors, ensemble, kernel_ridge, svm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import normalize
import xgboost as xgb

## Load data

In [3]:
# Ignore ORIGIN_CALL and other categorical columns
np.random.seed(42)
train_arr = np.loadtxt("../data/numeric_trainvf.csv", dtype=np.float32, delimiter=",", skiprows=1)[:, 4:]
mean = train_arr[:, -1].mean()
std = train_arr[:, -1].std()
train_arr = train_arr[train_arr[:, -1] < mean + 5 * std]
train_arr = train_arr[train_arr[:, -1] > 30]
# Split into training/validation sets
np.random.shuffle(train_arr)
valid_arr = train_arr[int(0.9 * train_arr.shape[0]):]
train_arr = train_arr[:int(0.9 * train_arr.shape[0])]
weights = train_arr[:, -1]

train_label = train_arr[:, -1]
train = train_arr[:, :-1]

valid_label = valid_arr[:, -1]
valid = valid_arr[:, :-1]


In [6]:
def score(model):
    pred = model.predict(valid)
    print(f"RMSE: {np.sqrt(np.sum((pred - valid_label)**2) / len(valid))}")

## Models

In [4]:
lin_reg = LinearRegression()
lin_reg.fit(train, train_label)
score(lin_reg)

RMSE: 438.2685160161196


### KNN Regressor

In [5]:
KNN_Regressor = neighbors.KNeighborsRegressor(n_neighbors=10)
KNN_Regressor.fit(train, train_label)

KNeighborsRegressor(n_neighbors=10)

In [6]:
score(KNN_Regressor)

RMSE: 445.49759614025527


In [7]:
np.std(KNN_Regressor.predict(valid))

187.77357

### Gradient Boosting

In [9]:
grad_boost = ensemble.GradientBoostingRegressor(learning_rate=0.001, n_estimators=200)
grad_boost.fit(train, train_label, sample_weight=train_label)

GradientBoostingRegressor(learning_rate=0.001, n_estimators=200)

In [10]:
score(grad_boost)

RMSE: 517.5286449968111


### Random Forest Regression

In [11]:
random_forest = ensemble.RandomForestRegressor(n_estimators=200, n_jobs=2, max_depth=5)
random_forest.fit(train, train_label)

RandomForestRegressor(max_depth=5, n_estimators=200, n_jobs=2)

In [12]:
score(random_forest)

RMSE: 431.8290644444473


### XGBoost

In [11]:
reg = xgb.XGBRegressor(tree_method="gpu_hist", n_estimators=100, max_depth=5)
reg.fit(train, train_label)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
import pandas as pd
test = np.loadtxt("../data/numeric_testvf.csv", delimiter=",", skiprows=1)[:, 3:]
preds = reg.predict(test)
df_pred = pd.read_csv("../data/sampleSubmission.csv")
df_pred["TRAVEL_TIME"] = preds
df_pred.to_csv("./xgboost_numeric_unweighted_no_prune_100_5.csv", index=False)

In [15]:
pred = reg.predict(valid[valid[:, 5] == 1])
print(f"RMSE: {np.sqrt(np.sum((pred - valid_label[valid[:, 5] == 1])**2) / len(valid[valid[:, 5] == 1]))}")
pred = reg.predict(valid[valid[:, 6] == 1])
print(f"RMSE: {np.sqrt(np.sum((pred - valid_label[valid[:, 6] == 1])**2) / len(valid[valid[:, 6] == 1]))}")
pred = reg.predict(valid[valid[:, 7] == 1])
print(f"RMSE: {np.sqrt(np.sum((pred - valid_label[valid[:, 7] == 1])**2) / len(valid[valid[:, 7] == 1]))}")

RMSE: 648.5610718712114
RMSE: 765.8376189759441
RMSE: 632.4334680943488


## Create prediction csv

In [16]:
import pandas as pd
test = np.loadtxt("../data/numeric_testvf.csv", delimiter=",", skiprows=1)[:, 3:]
df_pred = pd.read_csv("../data/sampleSubmission.csv")
# preds = lin_reg.predict(test)
# preds = KNN_Regressor.predict(test)
# preds = grad_boost.predict(test)

# df_pred["TRAVEL_TIME"] = preds
# df_pred.to_csv("./grad_boost_numeric_prune.csv", index=False)
# preds = random_forest.predict(test)

# df_pred["TRAVEL_TIME"] = preds
# df_pred.to_csv("./random_forest_numeric_prune.csv", index=False)
# preds = reg.predict(test)

df_pred["TRAVEL_TIME"] = preds
df_pred.to_csv("./xgboost_numeric_prune.csv", index=False)

In [13]:
print(preds)

[ 858.68787  790.6242   858.68787  769.01306  791.0932   860.5886
  858.68787  647.5201   910.2606   858.68787  716.3786   903.41797
 1095.327    837.9578   899.7239  1015.648    900.9468  1347.3323
  858.68787 1107.2839   858.68787  790.805   1115.9558   858.68787
  772.871    796.76733  858.68787  858.68787  797.6424   858.68787
  858.68787  790.6242   932.50616  869.53094 1408.8387   961.11584
  804.2347   752.72235  858.68787  782.3307   764.27014  922.26794
 1045.7333   910.2606   781.1745   794.23785  877.8405  1137.2795
  657.6205   716.3786   780.4091   765.66376  752.72235  858.68787
  858.68787  751.7747   910.2606   729.01117  943.3824   722.3846
  858.68787 1045.7333  1037.3832  1173.6993   910.2606   910.2606
  858.68787  886.645    884.403    780.4091  1091.1735   828.4643
  858.68787 1277.1282   607.6983  1094.2213   641.7769  1240.048
 1306.2747   889.2361   753.21136  907.2129   772.08167  733.8451
  894.0468   669.02344 1530.4978  1009.44495  646.6344  1832.2987
  742

In [ ]:
df_pred["TRAVEL_TIME"] = preds
df_pred.to_csv("./predictions.csv", index=False)